In [1]:
import pandas as pd
import numpy as np

In [2]:
y = pd.read_csv("cls_labelWithfea.csv")
print(len(y.loc[y["label"] == 0]))
print(len(y.loc[y["label"] == 1]))
y = y["label"]

32827
2418


In [3]:
fea_concat = pd.read_csv("sym_mesh_concat.csv",index_col=0)
print(fea_concat.shape)
fea_concat.head()

(35245, 692)


,0,1,2,3,4,5,6,7,8,9,...,38.1.1,39.1.1,40.1.1,41.1.1,42.1.1,43.1.1,44.1.1,45.1.1,46.1.1,47.1.1
0,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.646934,16.806405,21.451611,11.759815,115.787869,2.653693,93.246009,2261.805817,5.061076,18.577704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [4]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve

from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score

from sklearn.utils import shuffle

from imblearn.under_sampling import RandomUnderSampler # Random UnderSampler
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import roc_curve,auc

In [5]:
import random as python_random
seed = 1
np.random.seed(seed) # This is necessary for starting Numpy generated random numbers in a well-defined initial state.
python_random.seed(seed)# This is necessary for starting core Python generated random numbers in a well-defined state.

rus = RandomUnderSampler(sampling_strategy=1,random_state=seed) 
rus_x, rus_y = rus.fit_resample(fea_concat ,y) #rus_y(Series)
print('负采样后数据集大小:',len(rus_y))

fea_train,fea_test,y_train,y_test=train_test_split(
                rus_x,
                rus_y,
                test_size=0.2, shuffle=True)

print(fea_train.shape,fea_test.shape)

负采样后数据集大小: 4836
(3868, 692) (968, 692)


## MLP

### 5-fold

In [7]:
from sklearn.neural_network import MLPClassifier

kf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=seed)
    
test_auc_fold = []
test_aupr_fold = []
test_acc_fold= []

i=0
for train_index, test_index in kf.split(fea_train,y_train):
    X_train, Y_train = fea_train.iloc[train_index],y_train.iloc[train_index]
    X_test, Y_test = fea_train.iloc[test_index], y_train.iloc[test_index]

    clf = MLPClassifier(solver='adam',
                        hidden_layer_sizes=(128,64,64,32,32,16), 
                        activation='relu',
                        batch_size=32,
                        early_stopping=True,n_iter_no_change=5,
                        shuffle=True,
                        random_state=seed,
                       verbose=True)
    
    clf.fit(X_train, Y_train)
    
    Y_pred = clf.predict(X_test)
    
    test_auc = roc_auc_score(Y_test, Y_pred)#计算AUROC
    test_aupr = average_precision_score(Y_test, Y_pred)#计算AUPR
    test_acc = accuracy_score(Y_test, Y_pred)
    
    test_auc_fold.append(test_auc)
    test_aupr_fold.append(test_aupr)
    test_acc_fold.append(test_acc)

    print("{} iter".format(i))
    print('AUROC:',test_auc_fold[-1],'-----\t AUPR:',test_aupr_fold[-1],'-----\t ACC:',test_acc_fold[-1])
    i+=1

test_auc_fold.append(np.mean(test_auc_fold))
test_aupr_fold.append(np.mean(test_aupr_fold))
test_acc_fold.append(np.mean(test_acc_fold))

print('\n AverageAUC:')
print(test_auc_fold[-1])
print('AverageAUPR')
print(test_aupr_fold[-1])
print('AverageACC')
print(test_acc_fold[-1])

Iteration 1, loss = 1.32817634
Validation score: 0.503226
Iteration 2, loss = 0.90672997
Validation score: 0.600000
Iteration 3, loss = 0.83054508
Validation score: 0.587097
Iteration 4, loss = 0.72952846
Validation score: 0.535484
Iteration 5, loss = 0.68545160
Validation score: 0.587097
Iteration 6, loss = 0.62099954
Validation score: 0.616129
Iteration 7, loss = 0.58779462
Validation score: 0.606452
Iteration 8, loss = 0.61798717
Validation score: 0.606452
Iteration 9, loss = 0.56378552
Validation score: 0.583871
Iteration 10, loss = 0.55790108
Validation score: 0.622581
Iteration 11, loss = 0.55656832
Validation score: 0.625806
Iteration 12, loss = 0.51044494
Validation score: 0.641935
Iteration 13, loss = 0.52614747
Validation score: 0.641935
Iteration 14, loss = 0.50840206
Validation score: 0.635484
Iteration 15, loss = 0.48697612
Validation score: 0.587097
Iteration 16, loss = 0.47657935
Validation score: 0.648387
Iteration 17, loss = 0.46041657
Validation score: 0.638710
Iterat

- AverageAUC:
0.667866190017138
- AverageAUPR
0.6161433507922733
- AverageACC
0.6677865024686529

### Test

In [8]:
#y_test-binary-int;y_pred-proba-float
def test_metric(y_test,y_pred):
    y_test=np.array(y_test)
    y_pred=np.array(y_pred)
    
    y_pred_bina = np.around(y_pred,0).astype(int)
    class_acc=accuracy_score(y_test,y_pred_bina)
    
    y_test_int = y_test.astype(int)
    fpr,tpr,threshold=roc_curve(y_test_int,y_pred)#y_test-int(ndarray)
    print(len(fpr))
    roc_auc=auc(fpr,tpr)
    
    print('accuracy {}\t'.format(str(class_acc)), 'AUC:{}\n'.format(roc_auc))

    return np.array(class_acc),np.array(roc_auc)


In [9]:

clf = MLPClassifier(solver='adam',
                    hidden_layer_sizes=(128,64,64,32,32,16), 
                    activation='relu',
                    batch_size=32,
                    early_stopping=True,n_iter_no_change=5,
                    shuffle=True,
                    random_state=seed,
                    verbose=True)

clf.fit(fea_train,y_train)

predictions=[i[1] for i in clf.predict_proba(fea_test)]

accuracy,AUC = test_metric(y_test,predictions)

print('predictions lenth：{}\n'.format(len(predictions)),
    'accuracy {} \n'.format(accuracy),
     'AUC {} \n'.format(AUC))


Iteration 1, loss = 1.31610405
Validation score: 0.589147
Iteration 2, loss = 0.89786322
Validation score: 0.638243
Iteration 3, loss = 0.72943816
Validation score: 0.638243
Iteration 4, loss = 0.67481574
Validation score: 0.643411
Iteration 5, loss = 0.69089331
Validation score: 0.609819
Iteration 6, loss = 0.67501682
Validation score: 0.687339
Iteration 7, loss = 0.64267132
Validation score: 0.594315
Iteration 8, loss = 0.63559995
Validation score: 0.651163
Iteration 9, loss = 0.62310684
Validation score: 0.664083
Iteration 10, loss = 0.57725026
Validation score: 0.656331
Iteration 11, loss = 0.54649529
Validation score: 0.671835
Iteration 12, loss = 0.50507873
Validation score: 0.710594
Iteration 13, loss = 0.49624449
Validation score: 0.697674
Iteration 14, loss = 0.47768946
Validation score: 0.739018
Iteration 15, loss = 0.45365401
Validation score: 0.689922
Iteration 16, loss = 0.42648106
Validation score: 0.715762
Iteration 17, loss = 0.43560169
Validation score: 0.666667
Iterat

- accuracy 0.6673553719008265 
- AUC 0.7316970007690335 

In [10]:
AUC_data = np.array([list(y_test),list(predictions)])
print(AUC_data.shape)
AUC_data_df=pd.DataFrame(AUC_data.T,columns=["label","predict"])
AUC_data_df.head()

(2, 968)


,label,predict
0,0.0,0.057589
1,0.0,0.479983
2,0.0,0.894070
3,1.0,0.850448
4,0.0,0.707215


In [11]:
# AUC_data_df.to_csv("auc_data_mlp.csv")